In [ ]:
import sys
import torch
sys.path.append("../src")
from interpret import Abs_interpreter
from domain import Interval

In [ ]:
model = torch.load("../trained_models/pinn-burgers.pt")
domain = Interval()
abs_interpreter = Abs_interpreter(model, domain)
val_bounds, deriv_bounds = abs_interpreter.forward_pass(0, 1)
print("Val bounds:", val_bounds)
print("Deriv. bounds:", deriv_bounds)